In [ ]:
# Unzipper datasættet hvis det er en zipfil
!unzip data.zip

In [ ]:
# Montere Google drev som en mappe
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Installerer nødvendige biblioteker
!pip install tflite-model-maker
!pip install pycocotools

In [4]:
# Importerer nødvendige biblioteker
import os
import numpy as np
import tensorflow as tf

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

In [5]:
# Specificere stien for træning billeder og notationer
train_dir = os.path.join('data', 'training')
train_image_dir = os.path.join(train_dir, 'images')
train_annotation_dir = os.path.join(train_dir, 'annotations')

# Specificere stien for validerings billeder og notationer
val_dir = os.path.join('data', 'validation')
val_image_dir = os.path.join(val_dir, 'images')
val_annotation_dir = os.path.join(val_dir, 'annotations')

# Specificere stien for test billeder og notationer
test_dir = os.path.join('data', 'testing')
test_image_dir = os.path.join(test_dir, 'images')
test_annotation_dir = os.path.join(test_dir, 'annotations')

In [6]:
# Henter den mest præcise model, som bruges til at træne på
# (https://www.tensorflow.org/lite/models/modify/model_maker/object_detection#quickstart)
spec = model_spec.get('efficientdet_lite4')

In [7]:
# Label til alle objekter der skal genkendes
# Format: {x: 'label'}, hvor x er tal der stiger
# og "label" er navnet på objektet der skal genkendes

label_map = {1: 'Hjul'}

# Loader datasættet til Tensorflow 
train_data = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map=label_map)
val_data = object_detector.DataLoader.from_pascal_voc(val_image_dir, val_annotation_dir, label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_annotation_dir, label_map=label_map)

In [ ]:
# Koden der træner modellen
model = object_detector.create(train_data, model_spec=spec, epochs=40, batch_size=4, validation_data=val_data)

In [ ]:
# Ikke nødvendigt
# Evaluerer modellen og viser hvor præcis den er
model.evaluate(test_data)

In [ ]:
model.export(export_dir='.') # resulterer i model.tflite, som er den færdige model